# Certificate Revocation Dashboard

This dashboard visualizes certificate issuance and revocation activity across the lab's **triple-PKI infrastructure**. It parses structured log files written by the EDA revocation and issuance playbooks.

## Certificate Lifecycle

Every end-to-end test (`./lab test`) follows this lifecycle:

```
1. ISSUE     lab CLI calls pki-cli.py → Dogtag REST API → certificate issued
2. TRIGGER   Mock EDR/SIEM publishes event to Kafka (security-events topic)
3. ROUTE     Event-Driven Ansible matches event_type + pki_type → selects playbook
4. REVOKE    Playbook runs pki-cli.py revoke via SSH → Dogtag marks cert REVOKED
5. VERIFY    lab CLI polls certificate status until REVOKED (typically 10-20s)
```

Each step is logged. This dashboard reads the **revocation** and **issuance** logs to show what has happened and when.

## What triggers revocations?

Any of the 26 security event types can trigger revocation. Events arrive on Kafka from the Mock EDR or SIEM, and EDA routes them to the correct Dogtag CA based on `pki_type` (RSA / ECC / PQC) and `event_type` (determines CA level). Common triggers include key compromise, ransomware detection, credential theft, and IoT device cloning.

## Log Files

| Log | Path | Written By |
|-----|------|------------|
| **Revocation Log** | `/home/jovyan/logs/dogtag-revocation.log` | EDA revocation playbooks |
| **Issuance Log** | `/home/jovyan/logs/dogtag-issuance.log` | EDA issuance playbooks |

Both are pipe-delimited text files. If no logs exist yet, run `./lab test --pki-type rsa` from the host terminal to generate activity.

## PKI Architecture

The lab runs three independent Dogtag PKI hierarchies, each with a full CA chain:

```
                    ┌──────────────┐
                    │   Root CA    │  (self-signed)
                    └──────┬───────┘
                           │
                  ┌────────┴────────┐
                  │ Intermediate CA │  (signed by Root)
                  └───┬──────┬──────┘
                      │      │
           ┌──────────┤      ├──────────┐
           │          │      │          │
      ┌────┴───┐ ┌───┴───┐ ┌┴──────┐ ┌─┴──────┐
      │ IoT CA │ │EST CA │ │ACME CA│ │  ...   │
      └────────┘ └───────┘ └───────┘ └────────┘
```

| PKI Type | Algorithm | CA Levels | Revocation Playbook |
|----------|-----------|-----------|---------------------|
| **RSA-4096** | SHA-512 with RSA | root, intermediate, iot, est, acme | `dogtag-rsa-revoke-certificate.yml` |
| **ECC P-384** | ECDSA with SHA-384 | root, intermediate, iot, est | `dogtag-ecc-revoke-certificate.yml` |
| **ML-DSA-87** | NIST FIPS 204 Level 5 | root, intermediate, iot, est | `dogtag-pqc-revoke-certificate.yml` |

Revocation playbooks call `pki-cli.py revoke` which runs the Dogtag `pki` CLI inside the target CA container via `sudo podman exec`.

In [ ]:
import os
import re
import time
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

In [ ]:
# Log file paths
LOG_DIR = Path('/home/jovyan/logs')
REVOCATION_LOG = LOG_DIR / 'dogtag-revocation.log'
ISSUANCE_LOG = LOG_DIR / 'dogtag-issuance.log'

print(f"Log Directory: {LOG_DIR}")
print(f"Directory exists: {LOG_DIR.exists()}")
if LOG_DIR.exists():
    print(f"Contents: {list(LOG_DIR.iterdir())}")

## Log Parsing Functions

### Log Format

Both log files use a **pipe-delimited key-value format**. Each line looks like:

```
2025-01-15T10:30:00Z | PKI=rsa | CA=iot | SERIAL=0x1A2B3C | CN=sensor01.cert-lab.local | STATUS=REVOKED | REASON=key_compromise | EVENT=evt-abc123
```

| Field | Description | Values |
|-------|-------------|--------|
| `timestamp` | ISO 8601 timestamp (first field, no key prefix) | `2025-01-15T10:30:00Z` |
| `PKI` | PKI hierarchy type | `rsa`, `ecc`, `pqc` |
| `CA` | CA level that issued/revoked the cert | `root`, `intermediate`, `iot`, `est`, `acme` |
| `SERIAL` | Certificate serial number (hex) | `0x1A2B3C` |
| `CN` | Certificate Common Name (subject) | `device.cert-lab.local` |
| `STATUS` | Operation result | `REVOKED`, `ISSUED`, `FAILED` |
| `REASON` | Revocation reason (revocation log only) | `key_compromise`, `ca_compromise`, etc. |
| `EVENT` | Event ID that triggered this action | `evt-abc123` |

The parser below splits each line on ` | ` and extracts key-value pairs into a pandas DataFrame.

In [ ]:
def parse_log_line(line):
    """Parse a log line in the format: timestamp | KEY=VALUE | KEY=VALUE ..."""
    parts = line.strip().split(' | ')
    if len(parts) < 2:
        return None
    
    record = {'timestamp': parts[0]}
    for part in parts[1:]:
        if '=' in part:
            key, value = part.split('=', 1)
            record[key] = value
    return record

def read_log_file(log_path):
    """Read and parse a log file into a DataFrame."""
    if not log_path.exists():
        return pd.DataFrame()
    
    records = []
    with open(log_path, 'r') as f:
        for line in f:
            record = parse_log_line(line)
            if record:
                records.append(record)
    
    return pd.DataFrame(records)

def format_status(status):
    """Return HTML-formatted status with color."""
    colors = {
        'REVOKED': '#dc3545',  # red
        'ISSUED': '#28a745',   # green
        'FAILED': '#ffc107',   # yellow
    }
    color = colors.get(status, '#6c757d')
    return f'<span style="color: {color}; font-weight: bold;">{status}</span>'

## Revocation Log

In [ ]:
def display_revocations():
    df = read_log_file(REVOCATION_LOG)
    if df.empty:
        print("No revocation records found.")
        print(f"Log file: {REVOCATION_LOG}")
        return
    
    # Select and order columns
    columns = ['timestamp', 'PKI', 'CA', 'SERIAL', 'CN', 'STATUS', 'REASON', 'EVENT']
    available = [c for c in columns if c in df.columns]
    df_display = df[available].copy()
    
    # Sort by timestamp descending
    df_display = df_display.sort_values('timestamp', ascending=False)
    
    print(f"Total Revocations: {len(df_display)}")
    display(df_display.head(20))

display_revocations()

### Revocation Reasons (RFC 5280)

The `REASON` field in the revocation log corresponds to the CRL reason codes defined in RFC 5280 Section 5.3.1. The lab's EDA playbooks set the reason based on the security event type.

| Reason Code | Description | Typical Event Types |
|-------------|-------------|---------------------|
| `key_compromise` | Private key has been exposed or stolen | key_compromise, credential_theft, ransomware |
| `ca_compromise` | The issuing CA's key has been compromised | rogue_ca |
| `affiliation_changed` | Subject's organizational affiliation changed | compliance_violation |
| `superseded` | Certificate has been replaced by a new one | certificate_misuse |
| `cessation_of_operation` | Subject no longer operates the service | service_account_abuse |
| `privilege_withdrawn` | Privileges granted by the certificate are revoked | privilege_escalation, unauthorized_access |
| `unspecified` | No specific reason given | malware_detection, c2_communication, lateral_movement, and others |

Most lab scenarios use `key_compromise` as the default reason. The actual reason can be overridden in the EDA rulebook's `extra_vars`.

## Issuance Log

In [ ]:
def display_issuances():
    df = read_log_file(ISSUANCE_LOG)
    if df.empty:
        print("No issuance records found.")
        print(f"Log file: {ISSUANCE_LOG}")
        return
    
    columns = ['timestamp', 'PKI', 'CA', 'SERIAL', 'CN', 'STATUS', 'EVENT']
    available = [c for c in columns if c in df.columns]
    df_display = df[available].copy()
    df_display = df_display.sort_values('timestamp', ascending=False)
    
    print(f"Total Issuances: {len(df_display)}")
    display(df_display.head(20))

display_issuances()

## Summary Statistics

In [ ]:
def display_summary():
    revocations = read_log_file(REVOCATION_LOG)
    issuances = read_log_file(ISSUANCE_LOG)
    
    print("=" * 60)
    print("CERTIFICATE ACTIVITY SUMMARY")
    print("=" * 60)
    
    print(f"\nTotal Certificates Issued:  {len(issuances)}")
    print(f"Total Certificates Revoked: {len(revocations)}")
    
    if not revocations.empty and 'PKI' in revocations.columns:
        print("\nRevocations by PKI Type:")
        print(revocations['PKI'].value_counts().to_string())
    
    if not revocations.empty and 'CA' in revocations.columns:
        print("\nRevocations by CA:")
        print(revocations['CA'].value_counts().to_string())
    
    if not revocations.empty and 'STATUS' in revocations.columns:
        print("\nRevocation Status:")
        print(revocations['STATUS'].value_counts().to_string())

display_summary()

## PKI Breakdown Matrix

The table below cross-tabulates activity by **PKI type** (RSA / ECC / PQ) and **CA level** (root / intermediate / iot / est / acme). This shows where certificates are being issued and revoked across the full hierarchy.

In [ ]:
def display_pki_breakdown():
    revocations = read_log_file(REVOCATION_LOG)
    issuances = read_log_file(ISSUANCE_LOG)

    for label, df in [("Issuances", issuances), ("Revocations", revocations)]:
        print(f"\n{label}")
        print("-" * 50)
        if df.empty or 'PKI' not in df.columns or 'CA' not in df.columns:
            print(f"  No {label.lower()} data with PKI/CA columns.")
            continue

        pivot = pd.crosstab(df['PKI'], df['CA'], margins=True, margins_name='Total')
        # Reorder columns if possible
        col_order = [c for c in ['root', 'intermediate', 'iot', 'est', 'acme', 'Total'] if c in pivot.columns]
        pivot = pivot[col_order]
        display(pivot)

display_pki_breakdown()

## Live Dashboard

Auto-refreshing dashboard that polls logs every few seconds.

In [ ]:
def live_dashboard(refresh_interval=5, duration=120):
    """
    Display a live dashboard that auto-refreshes.
    
    Args:
        refresh_interval: Seconds between refreshes
        duration: Total duration to run (seconds)
    """
    start_time = time.time()
    
    while time.time() - start_time < duration:
        clear_output(wait=True)
        
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print(f"Certificate Revocation Lab Dashboard - {now}")
        print(f"Auto-refresh every {refresh_interval}s (running for {duration}s)")
        print("=" * 70)
        
        # Read logs
        revocations = read_log_file(REVOCATION_LOG)
        issuances = read_log_file(ISSUANCE_LOG)
        
        # Summary
        print(f"\n📊 SUMMARY")
        print(f"   Certificates Issued:  {len(issuances)}")
        print(f"   Certificates Revoked: {len(revocations)}")
        
        # Recent Activity
        print(f"\n📋 RECENT REVOCATIONS (last 5)")
        print("-" * 70)
        if not revocations.empty:
            recent = revocations.sort_values('timestamp', ascending=False).head(5)
            for _, row in recent.iterrows():
                ts = row.get('timestamp', 'N/A')[:19]
                pki = row.get('PKI', 'N/A')
                ca = row.get('CA', 'N/A')
                serial = row.get('SERIAL', 'N/A')[:16]
                cn = row.get('CN', 'N/A')[:30]
                status = row.get('STATUS', 'N/A')
                print(f"   {ts} | {pki:8} | {ca:12} | {serial:16} | {status}")
        else:
            print("   No revocations recorded yet.")
        
        print(f"\n📋 RECENT ISSUANCES (last 5)")
        print("-" * 70)
        if not issuances.empty:
            recent = issuances.sort_values('timestamp', ascending=False).head(5)
            for _, row in recent.iterrows():
                ts = row.get('timestamp', 'N/A')[:19]
                pki = row.get('PKI', 'N/A')
                ca = row.get('CA', 'N/A')
                serial = row.get('SERIAL', 'N/A')[:16]
                status = row.get('STATUS', 'N/A')
                print(f"   {ts} | {pki:8} | {ca:12} | {serial:16} | {status}")
        else:
            print("   No issuances recorded yet.")
        
        print("\n" + "=" * 70)
        elapsed = int(time.time() - start_time)
        remaining = duration - elapsed
        print(f"Remaining: {remaining}s | Press interrupt (⬛) to stop")
        
        time.sleep(refresh_interval)
    
    print("\nDashboard stopped.")

# Run dashboard for 2 minutes
live_dashboard(refresh_interval=5, duration=120)

## Certificate Lookup

Look up a certificate by serial number or CN.

In [ ]:
def lookup_certificate(search_term):
    """Search for a certificate in logs by serial or CN."""
    revocations = read_log_file(REVOCATION_LOG)
    issuances = read_log_file(ISSUANCE_LOG)
    
    results = []
    
    for df, log_type in [(issuances, 'ISSUED'), (revocations, 'REVOKED')]:
        if df.empty:
            continue
        
        # Search in SERIAL and CN columns
        mask = pd.Series([False] * len(df))
        if 'SERIAL' in df.columns:
            mask |= df['SERIAL'].str.contains(search_term, case=False, na=False)
        if 'CN' in df.columns:
            mask |= df['CN'].str.contains(search_term, case=False, na=False)
        
        matches = df[mask].copy()
        if not matches.empty:
            matches['ACTION'] = log_type
            results.append(matches)
    
    if results:
        combined = pd.concat(results, ignore_index=True)
        combined = combined.sort_values('timestamp')
        return combined
    else:
        return pd.DataFrame()

In [ ]:
# Example: Search for a certificate
# Change this to search for a specific certificate
SEARCH_TERM = "testdevice"

results = lookup_certificate(SEARCH_TERM)
if not results.empty:
    print(f"Found {len(results)} records matching '{SEARCH_TERM}':")
    display(results)
else:
    print(f"No records found matching '{SEARCH_TERM}'")

## Testing Workflow

To see this dashboard populate with real data, run `./lab test` from the host terminal. The test issues a certificate, triggers a security event, and waits for EDA to revoke it.

### Quick Start

```bash
# Single test — RSA key compromise (most common)
./lab test --pki-type rsa --scenario "Certificate Private Key Compromise"

# Test ECC hierarchy
./lab test --pki-type ecc --scenario "IoT Device Cloning Detected"

# Test post-quantum hierarchy
./lab test --pki-type pqc --scenario "Ransomware Encryption Detected"

# Test on the EST Sub-CA instead of IoT
./lab test --pki-type rsa --ca-level est

# Run all 26 scenarios
./lab test --pki-type rsa --all

# Run a category
./lab test --pki-type rsa --category iot
```

### Watching Results in Real-Time

1. Start the **Live Dashboard** cell above (it auto-refreshes every 5 seconds)
2. In a separate terminal, run `./lab test --pki-type rsa`
3. Watch the issuance appear first, then the revocation ~10-20 seconds later
4. Use the **Certificate Lookup** cell to search for the device name or serial number

### What to Expect

| Step | Timing | Log Entry |
|------|--------|-----------|
| Certificate issued | Immediate | Issuance log: `STATUS=ISSUED` |
| Security event published to Kafka | ~1 second | (visible in Notebook 01) |
| EDA matches event and runs playbook | ~5-10 seconds | — |
| Certificate revoked | ~10-20 seconds total | Revocation log: `STATUS=REVOKED` |

If revocation doesn't appear within 30 seconds, check EDA logs: `podman-compose logs eda-server`